In [13]:
#%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Definición de Librería

In [1]:
import tweepy
import json
import pandas as pd
import time
import GetOldTweets3 as got


In [2]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb


In [3]:
pd.set_option('display.max_rows', 1000)

In [4]:
#Definción de variables
df_followers_id = pd.DataFrame()
df_followers_info = pd.DataFrame()


# Definición de las claves para acceder a Twitter

In [5]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [6]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('max_colwidth',-1)

# Extracción de los datos de la cuenta

In [7]:
def extract_info_account():
    item = api.get_user('@elcorteingles')
    data_account=[]
    account={}
    account["name"]=item.name
    account["id"]=item.id
    account["screen name"]=item.screen_name
    account["friends_count"]=item.friends_count
    account["followers_count"]=item.followers_count
    data_account.append(account)
    df_account_info = pd.DataFrame(data_account)
    return df_account_info


# Followings

# Extracción de las cuentas Followings

In [8]:
#Nueva forma de obtener todas las cuentas sigue El Corte Inglés
#elcorteingles

# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_following_ids(api, twitter_account='@elcorteingles'):
    while True:
        try:
            for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
                following_ids.extend(page)
                #print(len(following_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return following_ids


# Definición de tramos de las cuentas Following

In [9]:
#Agrupación de ID por la limitación de la API de Twitter

#following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]


17


# Extracción información de las cuentas Followings

In [11]:
# Existe limitación al traer la información de los seguidores
def get_info_following(api,following_limit):
    data = []
    for followings in following_limit:
        following_obj=api.lookup_users(user_ids=followings) 
        for user in following_obj:
            following={}
            following["id"]=user.id
            following["name"]=user.name
            following["screen_name"]=user.screen_name
            following["created_at"]=user.created_at
            following["location"]=user.location
            following["followers_count"]=user.followers_count
            following["friends_count"]=user.friends_count
            following["description"]=user.description
            data.append(following)
    return data


,id,name,screen_name,created_at,location,followers_count,friends_count,description
0,252096294,Dani García,danigarcia_ca,2011-02-14 13:48:18,Marbella,126917,2971,"Cocinero desde los 17, 3*** Michelin y @Bibodanigarcia Marbella/Madrid/Doha/Tarifa @lobitodemardg Marbella/Madrid @hacerdecomertve"
1,940274906025091073,Javier Perez Pallares #Neumologia,jpallaresneumo,2017-12-11 17:39:41,"Cartagena, España",1932,867,#Neumologia. Interventional pulmonologist.Vicepresidente Asociacion Española Endoscopia Respiratoria y Neumologia Intervencionista. CEO @aerisrespira.
2,2236669338,Pedro Mozas Rello,pedromozasrello,2013-12-08 22:39:01,Madrid,303,376,"Publicista, escritor, guionista, blogger. Miembro del Club de Creativos de España. Autor de ""Incorrupto"" (Ed. Distrito 93 - @BooksBunker)."
3,570041347,Ana de Armas,Ana_d_Armas,2012-05-03 14:59:50,,238395,445,Artist.
4,1041739380455624704,alba reche,_albxreche,2018-09-17 17:23:17,España,208791,299,quimera https://t.co/YL3CeV0U9L
5,182764833,Metro de Madrid,metro_madrid,2010-08-25 10:18:32,"Madrid, España",559661,541,Metro de Madrid es la empresa pública que gestiona la red de metro de la Comunidad de Madrid.\r\nhttp://t.co/oKICiy53
6,200081186,Rocío Gil Grande,pepi_rocio,2010-10-08 12:02:33,Madrid,919,652,Periodista y feminista. Este perfil es personal.
7,1042007718553415682,Carlos Right,carlos_right,2018-09-18 11:09:34,"Barcelona, España",100143,244,#prisionero💘
8,197399746,Andy Lucas oficial,_AndyyLucas_,2010-10-01 11:58:43,España,115993,30049,"más de dos millones de discos vendidos,más de 100 premios nacionales e internacionales,más de mil conciertos en toda su carrera y cuatro millones de asistentes."
9,5494572,mtvspain,mtvspain,2007-04-25 11:30:20,"Madrid, Spain",2806194,914,"TV 📺 música 🎧 actualidad 🤗y más 🙌 Puedes vernos en Movistar+(24),Vodafone TV(153),Orange TV(60), Sky, Fubo TV, Virgin, Telecable(18),R(31), K(23) y #MTVPLAY"


In [12]:
def saveFile():
    file= File_Manage()
    fl_account_info = 'CSVs/info_el_corte_ingles.csv'
    file.setFile(fl_account_info)
    file.writeCSV(df_account_info,file.getFile())
    fl_following_id = 'CSVs/following_id_el_corte_ingles.csv'
    file.setFile(fl_following_id)
    file.writeCSV(df_following_id,file.getFile())
    fl_following_info = 'CSVs/following_info_el_corte_ingles.csv'
    file.setFile(fl_following_info)
    file.writeCSV(df_following_info,file.getFile())
